In [1]:
import tensorflow as tf

#### 1. 设定神经网络的参数

In [2]:
INPUT_NODE = 784 # 输入层节点数，图像像素大小
OUTPUT_NODE = 10 # 输出层节点数

IMAGE_SIZE = 28  # 图片大小
NUM_CHANNELS = 1 # 图片通道数，黑白的为1
NUM_LABELS = 10   # label数

CONV1_DEEP = 32  # 第一个卷积层深度，卷积核个数
CONV1_SIZE = 5  # 第一个卷积层卷积核大小

CONV2_DEEP = 64 #  第二个卷积层深度，卷积核个数
CONV2_SIZE = 5  # 第二个卷积层卷积核大小

FC_SIZE = 512  # 全连接层大小，节点个数

#### 2. 定义前向传播的过程

In [3]:
"""
正则表达式：输入层->(卷积层+->池化层？)+ ->全连接层+
LeNet-5: 输入层->卷积层->池化层->卷积层->池化层->全连接层->全连接层->输出层
"""
def inference(input_tensor, train, regularizer):  # 前向传播函数
    with tf.variable_scope('layer1-conv1'): # 2第一个卷积层，1是输入层
        conv1_weights = tf.get_variable(  
            "weight", [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))  # 权重
        conv1_biases = tf.get_variable("bias", [CONV1_DEEP], initializer=tf.constant_initializer(0.0)) # 偏置
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME') # 卷积
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases)) # relu激活函数

    with tf.name_scope("layer2-pool1"):  # 3第一个池化层
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME") # 池化

    with tf.variable_scope("layer3-conv2"): # 4第二个卷积层
        conv2_weights = tf.get_variable(
            "weight", [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1)) # 权重
        conv2_biases = tf.get_variable("bias", [CONV2_DEEP], initializer=tf.constant_initializer(0.0)) # 偏置
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME') # 卷积
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))  # relu激活函数

    with tf.name_scope("layer4-pool2"): # 5 第二个池化层
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # 池化
        pool_shape = pool2.get_shape().as_list() # 获取池化层的形状
        nodes = pool_shape[1] * pool_shape[2] * pool_shape[3] # 计算总的节点数
        reshaped = tf.reshape(pool2, [pool_shape[0], nodes]) # 压平，展成一维向量

    with tf.variable_scope('layer5-fc1'): # 6 第一个全连接层
        fc1_weights = tf.get_variable("weight", [nodes, FC_SIZE],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1)) # 权重
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights)) # 正则化项
        fc1_biases = tf.get_variable("bias", [FC_SIZE], initializer=tf.constant_initializer(0.1)) # 偏置

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases) # relu激活函数，relu(x*w + b)
        if train: fc1 = tf.nn.dropout(fc1, 0.5) # dropout正则化

    with tf.variable_scope('layer6-fc2'):  # 7 第二个全连接层，也是输出层 
        fc2_weights = tf.get_variable("weight", [FC_SIZE, NUM_LABELS],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))  # 权重
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))   # 正则化项
        fc2_biases = tf.get_variable("bias", [NUM_LABELS], initializer=tf.constant_initializer(0.1))  # 偏置
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases   # x*w + b

    return logit  # 返回输出